In [1]:
#%%
import numpy as np
import pdb
import itertools

In [2]:
#%% Helper function. poisson function
prob_space = {}
def poisson_prob(x,lam):
    # x - event occurring.
    # lam - expected value of the event.
    #pdb.set_trace()
    key = tuple(list((x,lam)))
    if key not in prob_space.keys():
        prob_space[key] = np.exp(-lam)*lam**x/np.math.factorial(x)
    return prob_space[key]

action_choice = np.arange(-5,6) 

In [3]:
#%%initialisations
class car_rental():
    def __init__(self, location_init, action_max):
        self.location = location_init
        self.action_space  = np.arange(action_max)
        self.num_state = len(location_init)
        self.max_req = 11
        self.max_ret = 11
      
        
    def reward(self, action, state_dict):
#        pdb.set_trace()
        tot_reward = 0
        for req_A in range(self.max_req):
            for req_B in range(self.max_req):
                # Joint probability of request.
                prob_req = poisson_prob(req_A,3)*poisson_prob(req_B,4)
                # Number of cars update.
                car_loc_A = min(req_A, self.location[0])
                car_loc_B = min(req_B, self.location[1])
                # Current reward.
                reward = 10*(car_loc_A + car_loc_B)
                
                car_loc_A = min(max(self.location[0] - car_loc_A - action,0),20)
                car_loc_B = min(max(self.location[1] - car_loc_B - action,0),20)
                
                reward = reward + (-2*np.abs(action))
                
                for ret_A in range(self.max_ret):
                    for ret_B in range(self.max_ret):
                        # Joint probability of return.
                        prob_ret = poisson_prob(ret_A,2)*poisson_prob(ret_B,3)
                        
                        prob_joint = prob_ret*prob_req
                        
                        car_loc_A = min(car_loc_A + ret_A,20)
                        car_loc_B = min(car_loc_B + ret_B,20)
                        
                        reward = prob_joint*(reward + \
                                             gamma*(state_dict[tuple(list((car_loc_A, car_loc_B)))]))
                        tot_reward = tot_reward + reward
        return tot_reward
        
    def ss_policy(self, action_all, state_dict):
#        pdb.set_trace()
        reward_action = []
        for i in action_all:
            tot_reward = 0
            prob_store = []
            for req_A in range(self.max_req):
                for req_B in range(self.max_req):
                    # Joint probability of request.
                    prob_req = poisson_prob(req_A,3)*poisson_prob(req_B,4)
                    # Number of cars update.
                    car_loc_A = min(req_A, self.location[0])
                    car_loc_B = min(req_B, self.location[1])
                    # Current reward.
                    reward = 10*(car_loc_A + car_loc_B)
                    
                    car_loc_A = min(max(self.location[0] - car_loc_A - i,0),20)
                    car_loc_B = min(max(self.location[1] - car_loc_B - i,0),20)
                    
                    reward = reward + (-2*np.abs(i))
                    
                    for ret_A in range(self.max_ret):
                        for ret_B in range(self.max_ret):
                            # Joint probability of return.
                            prob_ret = poisson_prob(ret_A,2)*poisson_prob(ret_B,3)
                            
                            prob_joint = prob_ret*prob_req
                            
                            car_loc_A = min(car_loc_A + ret_A,20)
                            car_loc_B = min(car_loc_B + ret_B,20)
                            prob_store.append(prob_joint)
                            reward = prob_joint*(reward + \
                                                 gamma*(state_dict[tuple(list((car_loc_A, car_loc_B)))]))
                            tot_reward = tot_reward + reward
            reward_action.append(tot_reward)
        return action_all[np.argmax(reward_action)]

In [4]:
#%% Poliy improvement and policy evaluation
# Possible action choice.
action_all = np.arange(-5,6)

# Discount factor.
gamma =0.9

# Error bound.
theta = 0.01

# Exit condition for loop.
loop = True

# Local counter.
cnt = 0

# Initialize all the states.
init_state = list(itertools.product(np.arange(21), repeat=2))

# Empty dictionary.
state = {}

for i in range(len(init_state)):
    state[init_state[i]] = 20
    
# Empty dictionary.
policy = {}

for i in range(len(init_state)):
    policy[init_state[i]] = 0

# Policy loop
policy_stable = False
    
# Policy counter 
plc_cnt = 0

while policy_stable== False:
    while loop == True:
        delta = 0.0
        
        for i in range(len(init_state)):
            
            action_choice = policy[init_state[i]]
            
            v = state[init_state[i]]
        
            agent_car = car_rental(init_state[i],20) #initial state
            
    #        pdb.set_trace()
            value_next_state = agent_car.reward(action_choice, state) 
            delta = max(delta, np.abs((v - value_next_state)))
            # print(delta)
            state[init_state[i]] = value_next_state
           
        cnt = cnt + 1
        print(cnt)
        if delta < theta:
            loop = False
        
    for i in range(len(init_state)):
        old_action = policy[init_state[i]]
        
        agent_car = car_rental(init_state[i],20) #initial state    
#        pdb.set_trace()
        pi_policy = agent_car.ss_policy(action_all, state)
        
        if old_action == pi_policy:
            policy_stable = True
        else:
            loop == True
            policy[init_state[i]] = pi_policy
        
    plc_cnt += 1
    print('Policy count = ', plc_cnt)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
Policy count =  1
